In [1]:
import torch
import clip
from torch.utils.data import DataLoader
from torchvision.datasets import CocoCaptions
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm
import os

In [2]:
# Инициализация устройства
device = "cuda" if torch.cuda.is_available() else "cpu"

# Загрузка архитектуры модели
model, processor = clip.load("ViT-B/32", device=device)

# Загрузка сохраненных весов
model.load_state_dict(torch.load("clip.pt", map_location=device))
model.to(device)
model.eval()

# Оптимизатор и функция потерь
tokenizer = clip.tokenize

In [3]:
transform = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
])

In [4]:
# Класс для работы с набором данных COCO
class CocoDataset(Dataset):
    def __init__(self, root, annFile, transform=None):
        super().__init__()
        self.coco = CocoCaptions(root=root, annFile=annFile)
        self.transform = transform  # Сохраняем transform

    def __len__(self):
        return len(self.coco)
        
    def __getitem__(self, idx):
        image, captions = self.coco[idx]
        caption = captions[0]
        if self.transform is not None:
            image = self.transform(image)  # Применяем transform
        return image, caption

In [5]:
# Загрузка тестового набора данных
BATCH_SIZE = 64
val_root_dir = "coco2017/val2017"  # Директория с валидационными изображениями
val_ann_file = "coco2017/annotations/captions_val2017.json"  # Файл аннотаций для валидации

In [6]:
# Проверка существования файла аннотаций
if not os.path.exists(val_ann_file):
    raise FileNotFoundError(f"Файл аннотаций не найден: {val_ann_file}")
else:
    print(f"Файл аннотаций найден: {val_ann_file}")

Файл аннотаций найден: coco2017/annotations/captions_val2017.json


In [7]:
# Создание DataLoader
test_dataset = CocoDataset(
    root=val_root_dir,
    annFile=val_ann_file,
    transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [8]:
# Проверка соответствия изображений и подписей
print("Проверка соответствия изображений и подписей:")
for i in range(3):  # Проверяем первые 3 примера
    image, caption = test_dataset[i]
    print(f"Пример {i + 1}:")
    print(f"  Подпись: {caption}")
    print(f"  Размер изображения: {image.shape}")
    print("-" * 50)

Проверка соответствия изображений и подписей:
Пример 1:
  Подпись: A woman stands in the dining area at the table.
  Размер изображения: torch.Size([3, 224, 224])
--------------------------------------------------
Пример 2:
  Подпись: A big burly grizzly bear is show with grass in the background.
  Размер изображения: torch.Size([3, 224, 224])
--------------------------------------------------
Пример 3:
  Подпись: Bedroom scene with a bookcase, blue comforter and window.
  Размер изображения: torch.Size([3, 224, 224])
--------------------------------------------------


In [9]:
# Функции для вычисления метрик
def recall_at_k(logits, targets, k=5):
    """
    Вычисляет Recall@K: долю правильных предсказаний среди топ-K.
    """
    _, top_k = logits.topk(k, dim=1)
    correct = top_k.eq(targets.view(-1, 1)).sum().item()
    return correct / len(targets)

In [10]:
# Оценка модели CLIP
model.eval()
results = {
    "correct": 0,  # Количество правильных предсказаний (для Recall@1)
    "total": 0,    # Общее количество примеров
    "recall@1": 0, # Recall@1
    "recall@5": 0, # Recall@5
    "mrr": 0       # Mean Reciprocal Rank
}

with torch.no_grad():
    for batch_idx, (images, texts) in enumerate(tqdm(test_loader)):
        # Проверка соответствия в батче (только для первого батча)
        if batch_idx == 0:  # Проверяем только первый батч
            print("Проверка соответствия в батче:")
            for i in range(min(3, len(images))):  # Проверяем первые 3 примера в батче
                print(f"  Изображение {i + 1}: {images[i].shape}")
                print(f"  Подпись {i + 1}: {texts[i]}")
                print("-" * 50)
                
        # Перенос данных на устройство (GPU/CPU)
        images = images.to(device)
        texts = tokenizer(texts, truncate=True).to(device)
        
        # Получаем выходы модели
        logits_per_image, logits_per_text = model(images, texts)  # Распаковываем оба выхода
        logits = logits_per_image  # Используем logits_per_image для изображение-текст
        
        # Проверка формы матрицы схожести
        assert logits.shape == (len(images), len(texts)), \
            f"Ожидается форма (batch_size, batch_size), получено {logits.shape}"

        predictions = logits.argmax(dim=1)
        targets = torch.arange(len(images)).to(device)
        
        # Основные метрики
        correct = (predictions == targets).sum().item()
        total = images.size(0)
        
        # Обновление счетчиков
        results["correct"] += correct
        results["total"] += total
        
        # Вычисление Recall@1 и Recall@5
        results["recall@1"] += correct  # Суммируем правильные ответы
        results["recall@5"] += recall_at_k(logits, targets, k=5) * total  # Суммируем с учетом размера батча
        
        # Вычисление Mean Reciprocal Rank (MRR)
        _, sorted_indices = logits.sort(descending=True)
        for i, target in enumerate(targets):
            rank = (sorted_indices[i] == target).nonzero().item() + 1
            results["mrr"] += 1.0 / rank

  0%|          | 0/79 [00:00<?, ?it/s]

Проверка соответствия в батче:
  Изображение 1: torch.Size([3, 224, 224])
  Подпись 1: A woman stands in the dining area at the table.
--------------------------------------------------
  Изображение 2: torch.Size([3, 224, 224])
  Подпись 2: A big burly grizzly bear is show with grass in the background.
--------------------------------------------------
  Изображение 3: torch.Size([3, 224, 224])
  Подпись 3: Bedroom scene with a bookcase, blue comforter and window.
--------------------------------------------------


100%|██████████| 79/79 [01:01<00:00,  1.29it/s]


In [11]:
# Расчет итоговых метрик
results["recall@1"] /= results["total"]  # Делим на общее количество примеров
results["recall@5"] /= results["total"]
results["mrr"] /= results["total"]

In [12]:
# Вывод результатов
print(f"Recall@1: {results['recall@1'] * 100:.2f}%")
print(f"Recall@5: {results['recall@5'] * 100:.2f}%")
print(f"Mean Reciprocal Rank (MRR): {results['mrr']:.4f}")

Recall@1: 86.86%
Recall@5: 99.18%
Mean Reciprocal Rank (MRR): 0.9236
